# This Notebook demonstrates the tool for Searching GBDX and ordering Imagery
This script was used for acquisition of Atlanta Data

In [3]:
from gbdxtools import Interface
gbdx = Interface()



In [4]:
# Specify Search Area for Atlanta
area_of_interest = "POLYGON((-84.44512435851775933 33.78293430345136983, -84.34661228447379244 33.78312166782949078, -84.33917315760092492 33.67325591918615402, -84.44782949556243068 33.67044181031717187, -84.44512435851775933 33.78293430345136983))"
start_date="2009-12-22T00:00:00.000Z"
end_date="2009-12-23T00:00:00.000Z"
results = gbdx.catalog.search(searchAreaWkt=area_of_interest, 
                              startDate=start_date, endDate=end_date)

In [5]:
import pandas as pd
# create dataframe from search results
catalog_df = pd.DataFrame([r['properties'] for r in results])
imagePrefix = "AOI_6_Atlanta/Atlanta_nadir{}_catid_{}"
catalog_df=catalog_df[catalog_df['platformName']=='WORLDVIEW02']
timeZ = catalog_df['timestamp'].sort_values()


In [6]:
## Save AOI Details to CSV.  
catalog_df[['catalogID','timestamp', 'panResolution','offNadirAngle', 'targetAzimuth']].sort_values(by=['timestamp']).reset_index().drop('index',axis=1).to_csv("AOI_Collect_Details.csv")

In [ ]:
orderList = []
workflowList = []
orderDictList = []

for idx, row in catalog_df.iterrows():
    
    # Use data from Catalog to create imagePrefix for saving
    catid = row['catalogID']
    output_location = imagePrefix.format(int(row['offNadirAngle']), catid)
    
    
    # Use gbdx ordering 
    order = gbdx.Task('Auto_Ordering')
    order.inputs.cat_id = catid
    order.impersonation_allowed=True
    
    # send GBDX Order
    orderDict = {'orderLocation': order.outputs.s3_location,
                'pan_ms_location': output_location,
                'catid': catid}
    
    order.outputs.s3_location
    
    ## Sent gbdx task
    aoptask = gbdx.Task('AOP_Strip_Processor', 
                        data=order.outputs.s3_location, 
                        enable_dra=False, 
                        ortho_epsg='UTM',
                       enable_pansharpen=False,
                       enable_acomp=True)
 
    workflow = gbdx.Workflow([order,aoptask])
    
    ## Save data
    workflow.savedata(aoptask.outputs.data, location=output_location)
    workflow.execute()
    
    # Store Order and Workflow to check status
    orderList.append(order)
    workflowList.append(workflow)
    
    orderDictList.append(orderDict)

import json
## Write order results for further query
with open('/raid/nfs/data/Datasets/CosmiQ_General_Study/AOI_6_Atlanta/order.json', 'w') as fout:
    json.dump(orderDictList, fout)

In [ ]:
## Print workflow status
for workflow in workflowList:
    print(workflow.status)
#for order_id in orderList:
#    gbdx.ordering.status(print(order_id))